<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:340 px; height:300px;">
<img src="https://raw.githubusercontent.com/unidata/drilsdown/master/docs/_static/DRILSDOWN_logo.png" alt="DRILSDOWN Logo" style="height: 300px;">
</div>

<div style="float:right; width:98 px; height:98px;">
<img src="https://www.earthcube.org/sites/default/files/doc-repository/logo_earthcube_cube-only_SMALL.png" alt="Earthcube Logo" style="height: 98px;">
</div>


<h1>Understanding eddy momentum flux III: Synthetic data</h1>
<h4>Suvarchal Cheedela and Brian Mapes, July 2018</h4>

## Kinematic construction of a cyclone in shear for KEdot elucidation
### Same array shape as a G5NR case study, overwritten as simple shear + deep Rankine vortex, optionally with zero-$\zeta_{abs}$ anticyclone on top 
#### + w field based on making vortex "resilient" (downshear ascent) -- for instance, a stretching term devised to oppose $\zeta$ tendencies due to advection by shearflow

Part of [this nbviewer repo](http://nbviewer.jupyter.org/github/brianmapes/G5NR_Interactive_notebooks/tree/master/)

-------------

<div style="clear:both"></div>
</div>

<hr style="height:2px;">


### Sections
1. <a href="#build">Build the synthetic u,v,w fields</a>
2. <a href="#display">display SKEdot exposition from those</a>

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs as ccrs
import xarray as xr
import numpy as np
from datetime import datetime
from holoviews import streams

import G5NR_utils

In [ ]:
%reload_ext ipython_IDV

# Grab the array shape from an arbitrary G5NR case study

In [ ]:
path = '/Users/bem/Jupyter/G5NR_Interactive_notebooks/'

#zbundlefile = path+'data/ZIDV_cases/skedot_40.1_prec_119.7_lat_28.6_lon_148.0_time_200610131230.zidv'
#xrdata_3d = xr.from_zidv(zbundlefile)

# Why the $&#(@#* are there NANs here? 
#print( xrdata_3d.u[0,70,:,128] )

In [ ]:
arrays = xr.open_dataset(path+'data/data_0_3D7km30minuteInst.nc')
#arrays
#print( arrays.u[0,70,:,128] ) # looks OK

<a name="build"></a>

# Shear flow: linear in pressure, zero at 500 hPa 

In [ ]:
ushear = arrays.u.copy()*0 + 10*(arrays.lev - 500.)/500.
vshear = arrays.v.copy()*0 

# Rankine vortex: ($v_{tan} \alpha r^{-1}$), except  ($v_{tan} \alpha r$) in inner core

In [ ]:
centerlat = arrays.lat.mean().values
centerlon = arrays.lon.mean().values

lon2d, lat2d = np.meshgrid(arrays.lon, arrays.lat)

# radius in degrees from center
r = np.sqrt( (lat2d-centerlat)**2 + (lon2d-centerlon)**2 )
az = np.arctan2((lat2d-centerlat), (lon2d-centerlon))

# vtan of 10 m/s at 1 degree, zero vorticity far field
vtan = 10.0/(r + 1e-6) # avoid division by zero; it will be overwritten in core anyway
# rankine near center
inner = np.where(r<1)
vtan[inner] = 10.0*r[inner]

vvortex =  vtan * np.cos(az)
uvortex = -vtan * np.sin(az)

# Broadcast vortex to all levels with p > 200 hPa

In [ ]:
ushearvort = ushear.copy()  # right shaped array
vshearvort = vshear.copy()

troposphere = np.where(arrays.lev > 200)
# print(troposphere)

for i in troposphere:
    ushearvort[:,i,:,:] += uvortex
    vshearvort[:,i,:,:] += vvortex

# Define a w field
## radial shape from vtan, wave1 in azimuth, sin(p) in vertical

In [ ]:
wshearvort = 0*ushearvort.copy() # right shaped array

left_of_downshear = np.pi/180.* 0 # angle offset of updraft from downshear direction

sinprofile = np.sin(np.pi * (arrays.lev - 200)/800.)

for i in troposphere:
    wshearvort[:,i,:,:] = vtan * np.cos(az - left_of_downshear) /10.  # max of 1 m/a
    wshearvort[:,i,:,:] *= sinprofile[i]

# Overwrite the u,v,w arrays 

In [ ]:
arrays.u.values = ushearvort
arrays.v.values = vshearvort
arrays.w.values = wshearvort

# select time, later it could be a slider but need to rewrite u_plot and v_plot for that
synth=arrays.isel(time=0) 

<a name="display"></a>

# OK, built the data. Now let's display it

In [ ]:
hv.notebook_extension('bokeh')
hv.archive.auto(exporters=[hv.Store.renderers['matplotlib'].instance(holomap=None)])

<a name="open_casefile"></a>

--------------
### Do the regridding to 4-degree (90,45 global grid) and deviations therefrom

In [ ]:
regrid_3d=G5NR_utils.regrid(synth,90,45)
subrid_3d=G5NR_utils.subgrid(synth,90,45)

skedot_da=G5NR_utils.SKEDot(synth.airdens,synth.u,synth.v,synth.w,90,45)

<a name="map_context"></a>

### Geoviews display for the subgrid scale filtered ("eddy") products:
#### level 50 is 525 mb

In [ ]:
up_img=gv.Dataset(subrid_3d.u[50,:,:]).to(gv.Image,kdims=['lon','lat'],label='up @525mb').redim.range(u=(-10,10))
vp_img=gv.Dataset(subrid_3d.v[50,:,:]).to(gv.Image,kdims=['lon','lat'],label='vp').redim.range(u=(-10,10))
wp_img=gv.Dataset(subrid_3d.w[50,:,:]).to(gv.Image,kdims=['lon','lat'],label='wp').redim.range(u=(-1,1))

In [ ]:
#subrid_3d.lev[50.:.:] #-->525 mb

In [ ]:
#%%output backend='matplotlib'
#%%opts Image (cmap='RdBu_r') [colorbar=True]
#%%opts Image (cmap='RdBu_r') [width=300 height=200 colorbar=True toolbar='above'] #for bokeh
#up_img*gf.coastline + vp_img*gf.coastline + wp_img*gf.coastline

In [ ]:
%%output backend='bokeh'
%%opts Image (cmap='RdBu_r') [width=300 height=200 colorbar=True xaxis=None, yaxis=None toolbar='above']
(up_img + vp_img + wp_img).cols(2)

--------------------
# Set up lineplots for profile interactive

In [ ]:
def u_plots(x,y):
    lon=x
    lat=y
    lvs=regrid_3d.u.sel(lat=lat,lon=lon,method='nearest').lev.values
    u=regrid_3d.u.sel(lat=lat,lon=lon,method='nearest').values
    upwp=skedot_da.upwp.sel(lat=lat,lon=lon,method='nearest').values
    uw=skedot_da.uw.sel(lat=lat,lon=lon,method='nearest').values
    udiv=skedot_da.Eddy_Tend_Zon.sel(lat=lat,lon=lon,method='nearest').values
    usheardiv=skedot_da.Eddy_Tend_Zon.sel(lat=lat,lon=lon,method='nearest').values*skedot_da.ushear.isel(lat=0,lon=0).values
    ubaro=skedot_da.ubaro.sel(lat=lat,lon=lon,method='nearest').values

    rho=regrid_3d.airdens.sel(lat=lat,lon=lon,method='nearest').values
    usheardiv_mean=np.nansum(usheardiv*rho)/np.nansum(rho)

    dp=lvs*100
    dpbyg=np.gradient(dp)/9.8 #MKS units: dM = dp/g
    skedot_zon=np.nansum(usheardiv*dpbyg)

    u_curve=hv.Curve((u, lvs), kdims=['U'], vdims=['pressure']).redim.range(pressure=(1000,0), U=(-10,10))
    u_curve=u_curve*hv.VLine(float(ubaro),label='ubaro')(style={'color':'black'})

    upwp_curve=hv.Curve((upwp, lvs), kdims=['UPWP[blue], UW[red]'], vdims=['pressure']).redim.range(pressure=(1000,0))
    upwp_curve=upwp_curve*hv.Curve((uw,lvs),kdims=['UW[red]'],vdims=['pressure']).redim.range(pressure=(1000,0))

    udiv_curve=hv.Curve((udiv, lvs), kdims=['div(rhoupwp)'], vdims=['pressure']).redim.range(pressure=(1000,0))
    udiv_curve=udiv_curve*hv.VLine(float(0),label='0')(style={'color':'black'})

    usheardiv_curve=hv.Curve((usheardiv, lvs), kdims=['div*ushear[skedot_zon='+format(skedot_zon,"0.2f")+']'], vdims=['pressure']).redim.range(pressure=(1000,0))
    usheardiv_curve=usheardiv_curve*hv.VLine(float(usheardiv_mean),label='umean')(style={'color':'black'})
    return (upwp_curve+udiv_curve+u_curve+usheardiv_curve)
    

In [ ]:
def v_plots(x,y):
    lon=x
    lat=y
    lvs=regrid_3d.v.sel(lat=lat,lon=lon,method='nearest').lev.values
    v=regrid_3d.v.sel(lat=lat,lon=lon,method='nearest').values
    vpwp=skedot_da.vpwp.sel(lat=lat,lon=lon,method='nearest').values
    vw=skedot_da.vw.sel(lat=lat,lon=lon,method='nearest').values
    vdiv=skedot_da.Eddy_Tend_Mer.sel(lat=lat,lon=lon,method='nearest').values
    vsheardiv=skedot_da.Eddy_Tend_Mer.sel(lat=lat,lon=lon,method='nearest').values*skedot_da.vshear.isel(lat=0,lon=0).values
    vbaro=skedot_da.vbaro.sel(lat=lat,lon=lon,method='nearest').values

    dp=lvs*100
    dpbyg=np.gradient(dp)/9.8 #MKS units: dM = dp/g
    skedot_mer=np.nansum(vsheardiv*dpbyg)

    rho=regrid_3d.airdens.sel(lat=lat,lon=lon,method='nearest').values
    vsheardiv_mean=np.nansum(vsheardiv*rho)/np.nansum(rho)


    v_curve=hv.Curve((v, lvs), kdims=['V'], vdims=['pressure']).redim.range(pressure=(1000,0), V=(-5,5))
    v_curve=v_curve*hv.VLine(float(vbaro),label='vbaro')(style={'color':'black'})

    vpwp_curve=hv.Curve((vpwp, lvs), kdims=['VPWP[blue], VW[red]'], vdims=['pressure']).redim.range(pressure=(1000,0))
    vpwp_curve=vpwp_curve*hv.Curve((vw,lvs),kdims=['VW[red]'],vdims=['pressure']).redim.range(pressure=(1000,0))

    vdiv_curve=hv.Curve((vdiv, lvs), kdims=['div(rhovpwp)'], vdims=['pressure']).redim.range(pressure=(1000,0))
    vdiv_curve=vdiv_curve*hv.VLine(float(0),label='0')(style={'color':'black'})

    vsheardiv_curve=hv.Curve((vsheardiv, lvs), kdims=['div*vshear[skedot_mer='+format(skedot_mer,"0.2f")+']'], vdims=['pressure']).redim.range(pressure=(1000,0))
    vsheardiv_curve=vsheardiv_curve*hv.VLine(float(vsheardiv_mean),label='vbaro')(style={'color':'black'})
    return (vpwp_curve+vdiv_curve+v_curve+vsheardiv_curve)

In [ ]:
def uv_plots(x,y):
    return u_plots(x,y)+v_plots(x,y)

In [ ]:
#skedot_da

In [ ]:
hvd=hv.Dataset(skedot_da.SKEDOT,kdims=['lon','lat'],vdims=['SKEDOT'])
skedot_img=hvd.to(hv.Image,kdims=['lon','lat'],vdims=['SKEDOT']).redim.range(SKEDOT=(-5,5))

In [ ]:
%%opts Image (cmap='RdBu_r') [width=600 height=400 colorbar=True toolbar='above' tools=['tap']]
tap=streams.SingleTap(source=skedot_img,x=regrid_3d.lon.values[0],y=regrid_3d.lat.values[0])
pointer=streams.PointerXY(source=skedot_img,x=regrid_3d.lon.values[0],y=regrid_3d.lat.values[0])
pointer_map=hv.DynamicMap(lambda x,y: hv.Points([(x,y)])(style={'size':10,'color':'black'}),streams=[pointer])

In [ ]:
u_dyn_plot=hv.DynamicMap(u_plots,kdims=[],streams=[tap])
v_dyn_plot=hv.DynamicMap(v_plots,kdims=[],streams=[tap])
uv_dyn_plot=hv.DynamicMap(uv_plots,kdims=[],streams=[tap])

<a name="profiles"></a>

-------------
## Profiles of the quantities behind the filterscale SKEdot map

In [ ]:
skedot_img*pointer_map

In [ ]:
%%opts Curve [width=200 show_grid=True]
uv_dyn_plot

In [ ]:
#%%output filename="lineplots" fig="png"
#uv_dyn_plot

In [ ]:
hv.archive.export()